In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
model_name = '1080ti'
file_name = '1080ti'

In [3]:
DATA_DIR = os.path.abspath("../../../prediction_model/data/raw_data")
SAVE_DIR = os.path.abspath("../../../prediction_model/data/")

## Dense Layer

In [4]:
dfDense = pd.read_pickle(os.path.join(DATA_DIR,'%s/1/benchmark_dense__20180925.pkl' %file_name))

for i in range(3,4):
    dfDense = pd.concat([dfDense,pd.read_pickle(os.path.join(DATA_DIR,'%s/%d/benchmark_dense__20180925.pkl' %(file_name, i)))])

In [5]:
ops = (dfDense['batchsize']
       * dfDense['dim_input'] 
       * dfDense['dim_output'])

memory_weights = dfDense['dim_input'] * dfDense['dim_output']
memory_in = dfDense['batchsize'] * dfDense['dim_input']
memory_out = dfDense['batchsize'] * dfDense['dim_output']


dfDense['optimizer'] = dfDense['optimizer'].replace({0:'opt_None',
                                                     1:'opt_SGD',
                                                     2:'opt_Adadelta',
                                                     3:'opt_Adagrad',
                                                     4:'opt_Momentum',
                                                     5:'opt_Adam',
                                                     6:'opt_RMSProp'})

dfDense['activation_fct'] = dfDense['activation_fct'].replace({0:'act_None',
                                                               1:'act_relu',
                                                               2:'act_tanh',
                                                               3:'act_sigmoid'})

dfDense['ops'] = ops
dfDense['memory_weights'] = memory_weights
dfDense['memory_in'] = memory_in
dfDense['memory_out'] = memory_out

In [6]:
one_hot_optimizer = pd.get_dummies(dfDense['optimizer'])

dfDense = dfDense.drop(labels='optimizer',axis=1)
dfDense = pd.concat([dfDense,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfDense['activation_fct'])

dfDense = dfDense.drop(labels='activation_fct',axis=1)
dfDense = pd.concat([dfDense,one_hot_activation],axis=1)

In [7]:
dfDense.describe()

,batchsize,dim_input,dim_output,gpu,precision,timeUsed_max,timeUsed_median,timeUsed_min,timeUsed_std,ops,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.000000,25000.000000,2.500000e+04,...,25000.000000,25000.00000,25000.000000,25000.00000,25000.000000,25000.00000,25000.000000,25000.000000,25000.00000,25000.00000
mean,20.974880,2050.941160,2050.187800,0.0,32.0,0.622732,0.577794,0.527819,0.035666,8.900494e+07,...,0.083320,0.08320,0.086160,0.49676,0.082280,0.08160,0.244120,0.250840,0.24972,0.25532
std,11.801843,1187.544563,1191.719451,0.0,0.0,0.526965,0.536531,0.524457,0.014407,1.033539e+08,...,0.276371,0.27619,0.280606,0.50000,0.274796,0.27376,0.429573,0.433505,0.43286,0.43605
min,1.000000,1.000000,1.000000,0.0,32.0,0.117497,0.104036,0.094442,0.001617,4.980000e+02,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000
25%,11.000000,1024.000000,1010.000000,0.0,32.0,0.267854,0.202231,0.169635,0.025396,1.430348e+07,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000
50%,21.000000,2059.000000,2049.000000,0.0,32.0,0.408139,0.351710,0.296450,0.037082,4.952736e+07,...,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.00000,0.00000
75%,31.000000,3082.000000,3088.000000,0.0,32.0,0.798798,0.771527,0.700685,0.045882,1.270239e+08,...,0.000000,0.00000,0.000000,1.00000,0.000000,0.00000,0.000000,1.000000,0.00000,1.00000
max,41.000000,4095.000000,4095.000000,0.0,32.0,3.343759,3.317218,3.268304,0.093956,6.615837e+08,...,1.000000,1.00000,1.000000,1.00000,1.000000,1.00000,1.000000,1.000000,1.00000,1.00000


In [8]:
dfDense.to_pickle(os.path.join(SAVE_DIR,'Data_dense_%s.pkl'%model_name))

## Convolutional Layers

In [9]:
dfConv = pd.read_pickle(os.path.join(DATA_DIR,'%s/0/benchmark_convolution__20180924.pkl' %file_name))

header = dfConv.columns

for i in range(2,3):
    dfConv = pd.concat([dfConv,pd.read_pickle(os.path.join(DATA_DIR,'%s/%i/benchmark_convolution__20180925.pkl' %(file_name, i)))])

In [10]:
padding_reduction = ((dfConv['padding']==0)
                     *(dfConv['kernelsize']-1))

elements_output = ((dfConv['matsize'] - padding_reduction)
                   / dfConv['strides'])**2

ops = (dfConv['batchsize']
       * elements_output
       * dfConv['kernelsize']**2
       * dfConv['channels_in']
       * dfConv['channels_out'])

memory_weights = (dfConv['kernelsize']**2
                  * dfConv['channels_in']
                  * dfConv['channels_out']
                  + dfConv['use_bias'] * dfConv['channels_out'])

memory_in = (dfConv['batchsize']
             * dfConv['matsize']**2
             * dfConv['channels_in'])

memory_out = (dfConv['batchsize']
              * elements_output
              * dfConv['channels_out'])


dfConv['elements_matrix'] = dfConv['matsize']**2
dfConv['elements_kernel'] = dfConv['kernelsize']**2

    
dfConv['ops'] = ops
dfConv['memory_weights'] = memory_weights
dfConv['memory_in'] = memory_in
dfConv['memory_out'] = memory_out


dfConv['optimizer'] = dfConv['optimizer'].replace({0:'opt_None',
                                                   1:'opt_SGD',
                                                   2:'opt_Adadelta',
                                                   3:'opt_Adagrad',
                                                   4:'opt_Momentum',
                                                   5:'opt_Adam',
                                                   6:'opt_RMSProp'})

dfConv['activation_fct'] = dfConv['activation_fct'].replace({0:'act_None',
                                                             1:'act_relu',
                                                             2:'act_tanh',
                                                             3:'act_sigmoid'})

dfConv['use_bias'] = np.uint8(dfConv['use_bias'])


dfConv.dropna(inplace=True)

In [11]:
one_hot_optimizer = pd.get_dummies(dfConv['optimizer'])

dfConv = dfConv.drop(labels='optimizer',axis=1)
dfConv = pd.concat([dfConv,one_hot_optimizer],axis=1)


one_hot_activation = pd.get_dummies(dfConv['activation_fct'])

dfConv = dfConv.drop(labels='activation_fct',axis=1)
dfConv = pd.concat([dfConv,one_hot_activation],axis=1)

In [12]:
dfConv.describe()

,batchsize,channels_in,channels_out,gpu,kernelsize,matsize,padding,precision,strides,timeUsed_max,...,opt_Adagrad,opt_Adam,opt_Momentum,opt_None,opt_RMSProp,opt_SGD,act_None,act_relu,act_sigmoid,act_tanh
count,25000.000000,25000.00000,25000.000000,25000.0,25000.000000,25000.000000,25000.000000,25000.0,25000.000000,25000.000000,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,20.832400,64.07096,64.071560,0.0,3.993280,255.977520,0.498440,32.0,2.494640,10.131081,...,0.080560,0.085640,0.086800,0.493880,0.087480,0.084000,0.247800,0.246800,0.253840,0.251560
std,11.820953,300.48600,305.070258,0.0,2.005621,147.529194,0.500008,0.0,1.114388,16.749052,...,0.272164,0.279837,0.281547,0.499973,0.282543,0.277393,0.431744,0.431158,0.435216,0.433918
min,1.000000,1.00000,1.000000,0.0,1.000000,1.000000,0.000000,32.0,1.000000,0.155821,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11.000000,10.00000,10.000000,0.0,2.000000,128.000000,0.000000,32.0,2.000000,1.379640,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,21.000000,19.00000,19.000000,0.0,4.000000,256.000000,0.000000,32.0,2.000000,3.676519,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,31.000000,39.00000,38.000000,0.0,6.000000,383.000000,1.000000,32.0,3.000000,11.258588,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,41.000000,9959.00000,9858.000000,0.0,7.000000,512.000000,1.000000,32.0,4.000000,244.702840,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
dfConv.to_pickle(os.path.join(SAVE_DIR,'Data_convolution_%s.pkl'%model_name))